## **Working With Data**

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download Training Data from Open Datasets
training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())

# Download Test Data from Open Datasets
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

In [3]:
# Define Batch Size (the no. of data points and lables in a group)
batch_size = 64

# Create Data Loaders
training_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### **Creating a Model**

In [4]:
# Accelerating Enabling
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### **Optimising a Model's Parameters**

In [ ]:
# To train a model, we need a Loss Function and an Optimiser.
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=1e-3)


# Training the Chosen Model
def train(dataloader, model, loss_fn, optimiser):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


# Check Performance agianst Test Dataset
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


# Train, Test, Adjust the Model Iterativly (epochs)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimiser)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302801  [   64/60000]
loss: 2.291761  [ 6464/60000]
loss: 2.274807  [12864/60000]
loss: 2.269343  [19264/60000]
loss: 2.243467  [25664/60000]
loss: 2.220306  [32064/60000]
loss: 2.226251  [38464/60000]
loss: 2.196064  [44864/60000]
loss: 2.193167  [51264/60000]
loss: 2.152608  [57664/60000]
Test Error: 
 Accuracy: 41.8%, Avg loss: 2.150220 

Epoch 2
-------------------------------
loss: 2.163800  [   64/60000]
loss: 2.157073  [ 6464/60000]
loss: 2.096115  [12864/60000]
loss: 2.112448  [19264/60000]
loss: 2.049086  [25664/60000]
loss: 1.996615  [32064/60000]
loss: 2.030274  [38464/60000]
loss: 1.949271  [44864/60000]
loss: 1.954061  [51264/60000]
loss: 1.872307  [57664/60000]
Test Error: 
 Accuracy: 52.9%, Avg loss: 1.872594 

Epoch 3
-------------------------------
loss: 1.908824  [   64/60000]
loss: 1.881834  [ 6464/60000]
loss: 1.758170  [12864/60000]
loss: 1.805871  [19264/60000]
loss: 1.677611  [25664/60000]
loss: 1.641065  [32064/600

### **Saving a Model**

In [6]:
# A common way to save a model is to serialise the internal state dictionary (containing the model parameters).
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### **Loading a Model**

In [7]:
# The process for loading a model includes re-creating the model structure and loading the state dictionary into it.
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

# Using the Model to Make Predictions
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
